In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
 !pip install pororo

     |████████████████████████████████| 266kB 6.6MB/s 
     |████████████████████████████████| 2.5MB 7.0MB/s 
     |████████████████████████████████| 3.1MB 28.0MB/s 
     |████████████████████████████████| 471kB 48.5MB/s 
     |████████████████████████████████| 748.8MB 20kB/s 
     |████████████████████████████████| 5.9MB 30.1MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 1.5MB 51.0MB/s 
     |████████████████████████████████| 276kB 56.7MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
     |████████████████████████████████| 1.7MB 46.7MB/s 
     |████████████████████████████████| 901kB 52.1MB/s 
     |████████████████████████████████| 3.3MB 54.4MB/s 
     |████████████████████████████████| 184kB 57.6MB/s 
     |████████████████████████████████| 1.2MB 42.0MB/s 
     |████████████████████████████████| 153kB 59.0MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 112kB 56.8MB/s 

In [3]:
!pip install konlpy


     |████████████████████████████████| 19.4MB 477kB/s 
     |████████████████████████████████| 450kB 52.6MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [21]:
from pororo import Pororo
from konlpy.tag import Okt
import os
import pandas as pd
from math import sqrt
from tqdm import tqdm_notebook as tqdm
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

**ex) 검색어가 flask 로 넘겨졌을 때**


In [11]:
okt = Okt()
word_list = list(okt.nouns(u'양자역학 이론에 대하여'))
word_list

['양자역학', '이론', '대하']

/content/gdrive/MyDrive/fasttext


**불용어 제거**  추후 수정이 필요


In [12]:
f = open('./한국어불용어100.txt','r')

In [13]:
stopword = list()

f_list =  f.readlines()
for i in f_list :
  stopword.append(i.split()[0])


In [14]:
for noun in word_list :
  for word in stopword :
    if word == noun :
      word_list.remove(word)

In [15]:
word_list # 명사부분으로만 검색할 수 있도록

['양자역학', '이론']

**충분한 데이터셋으로 추천피드 좋아요가 많이 눌렀을때**

In [16]:
rating_path = '/content/gdrive/MyDrive/fasttext/'
ratings_df = pd.read_csv(os.path.join(rating_path, 'ratings.csv'), encoding='utf-8')

print(ratings_df.shape)

(1048575, 4)


In [17]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=1234)

print(train_df.shape)
print(test_df.shape)

(838860, 4)
(209715, 4)


In [18]:
sparse_matrix = train_df.groupby('searchid').apply(lambda x: pd.Series(x['rating'].values, index=x['userId'])).unstack()
sparse_matrix.index.name = 'searchid'

sparse_matrix

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,7006,7007,7008,7009,7010,7011,7012,7013,7014,7015,7016,7017,7018,7019,7020,7021,7022,7023,7024,7025,7026,7027,7028,7029,7030,7031,7032,7033,7034,7035,7036,7037,7038,7039,7040,7041,7042,7043,7044,7045
searchid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,4.0,3.0,4.0,NaN,NaN,4.0,NaN,3.5,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,5.0,4.0,NaN,NaN,NaN,5.0,3.0,NaN,3.0,NaN,4.0,3.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,2.0,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,3.5,4.0,NaN,NaN,NaN,4.5
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**결측치값을 평균값으로 item 기반**

In [19]:
sparse_matrix_withsearch = sparse_matrix.apply(lambda x: x.fillna(x.mean()), axis=1)

In [20]:
sparse_matrix_withsearch.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,7006,7007,7008,7009,7010,7011,7012,7013,7014,7015,7016,7017,7018,7019,7020,7021,7022,7023,7024,7025,7026,7027,7028,7029,7030,7031,7032,7033,7034,7035,7036,7037,7038,7039,7040,7041,7042,7043,7044,7045
searchid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.915857,3.500000,4.000000,3.000000,4.000000,3.915857,3.915857,4.000000,3.915857,3.500000,3.915857,4.000000,3.915857,3.915857,3.915857,3.915857,3.915857,3.00000,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,3.915857,5.000000,3.915857,3.915857,3.915857,3.915857,...,5.000000,4.000000,3.915857,3.915857,3.915857,5.000000,3.000000,3.915857,3.000000,3.915857,4.000000,3.500000,3.915857,3.915857,3.915857,3.915857,3.915857,4.000000,3.915857,3.915857,3.915857,3.000000,2.000000,4.000000,3.915857,3.915857,3.915857,5.000000,3.915857,3.915857,3.915857,3.915857,3.915857,5.000000,3.500000,4.000000,3.915857,3.915857,3.915857,4.500000
2,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,5.000000,3.291720,3.291720,2.000000,3.291720,3.291720,3.291720,3.291720,3.291720,3.29172,3.291720,2.500000,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,...,3.291720,4.000000,3.291720,3.291720,3.291720,4.000000,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,2.500000,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720,3.291720
3,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,4.000000,3.154846,3.154846,3.154846,2.000000,3.154846,3.154846,3.154846,3.154846,3.154846,1.50000,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,...,3.154846,3.154846,3.154846,3.154846,3.154846,5.000000,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846,3.154846
4,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.96875,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,...,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750,2.968750
5,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,4.00000,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,...,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775,3.132775

**matrix 로 Cos 유사도 추출**

In [22]:
def cos_matrix(a, b):
    cos_values = cosine_similarity(a.values, b.values)
    cos_df = pd.DataFrame(data=cos_values, columns = a.index.values, index=a.index)

    return cos_df

In [25]:
search_cos_df = cos_matrix(sparse_matrix_withsearch, sparse_matrix_withsearch)
search_cos_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,205583,205661,205799,205945,205967,206068,206135,206208,206272,206293,206385,206435,206499,206523,206714,206727,206777,206805,206845,206861,206899,206949,207023,207111,207203,207213,207309,207311,207367,207405,207642,207830,207890,208002,208080,208112,208737,208793,208939,209163
searchid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.989574,0.989914,0.992296,0.990408,0.990402,0.991204,0.992768,0.991890,0.989427,0.990742,0.991397,0.992542,0.991752,0.991979,0.990826,0.990631,0.991422,0.983489,0.991983,0.989710,0.991312,0.991932,0.991099,0.988549,0.992517,0.992617,0.992487,0.991884,0.992536,0.991335,0.987697,0.992844,0.987389,0.992506,0.990380,0.992839,0.992756,0.988453,0.992664,...,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992858,0.992851,0.992851,0.992848,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992848,0.992851,0.992852,0.992850,0.992851,0.992847,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851,0.992851
2,0.989574,1.000000,0.992698,0.994873,0.993497,0.992874,0.993772,0.995391,0.994752,0.992237,0.993277,0.994147,0.995277,0.994297,0.994814,0.993578,0.993125,0.994339,0.987054,0.994818,0.992144,0.994426,0.994927,0.994092,0.991179,0.995078,0.995281,0.995149,0.994511,0.995234,0.994293,0.990325,0.995487,0.989212,0.995208,0.993214,0.995480,0.995386,0.990651,0.995335,...,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995469,0.995490,0.995490,0.995488,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995487,0.995490,0.995485,0.995480,0.995490,0.995487,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490,0.995490
3,0.989914,0.992698,1.000000,0.996068,0.994365,0.993901,0.995040,0.996487,0.996013,0.992746,0.994368,0.995381,0.996304,0.995317,0.995697,0.994377,0.994064,0.995434,0.987543,0.995938,0.992909,0.995100,0.995984,0.994894,0.991880,0.996255,0.996339,0.996215,0.995405,0.996333,0.995018,0.990729,0.996581,0.989473,0.996279,0.994007,0.996563,0.996462,0.991192,0.996466,...,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996579,0.996594,0.996594,0.996592,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996591,0.996594,0.996592,0.996591,0.996594,0.996591,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594,0.996594
4,0.992296,0.994873,0.996068,1.000000,0.996714,0.996378,0.997273,0.999169,0.998347,0.995094,0.996625,0.997974,0.998900,0.998144,0.998442,0.997046,0.996699,0.997927,0.989501,0.998377,0.995426,0.997662,0.998450,0.997467,0.994817,0.998854,0.999032,0.998960,0.998272,0.998988,0.997795,0.993448,0.999277,0.991961,0.999008,0.996525,0.999268,0.999177,0.993966,0.999112,...,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999263,0.999279,0.999279,0.999277,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999276,0.999279,0.999277,0.999276,0.999279,0.999276,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279,0.999279
5,0.990408,0.993497,0.994365,0.996714,1.000000,0.994402,0.995718,0.997135,0.996396,0.993334,0.994914,0.995987,0.996839,0.996156,0.996420,0.995102,0.994740,0.995880,0.987655,0.996302,0.993423,0.995749,0.996444,0.995387,0.992560,0.996866,0.996925,0.996876,0.996197,0.996882,0.995927,0.991425,0.997219,0.990127,0.996983,0.994597,0.997217,0.997100,0.991975,0.997029,...,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997210,0.997221,0.997221,0.997219,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997221,0.997218,0

In [24]:
userId_grouped = train_df.groupby('userId')
search_prediction_ = pd.DataFrame(index=list(userId_grouped.indices.keys()), columns=sparse_matrix_withsearch.index)
search_prediction_

searchid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,205583,205661,205799,205945,205967,206068,206135,206208,206272,206293,206385,206435,206499,206523,206714,206727,206777,206805,206845,206861,206899,206949,207023,207111,207203,207213,207309,207311,207367,207405,207642,207830,207890,208002,208080,208112,208737,208793,208939,209163
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
for userId, group in tqdm(userId_grouped):
    
    user_sim = search_cos_df.loc[group['searchid']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)

    # userId의 전체 rating predictions 
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    search_prediction_.loc[userId] = pred_ratings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


**모델을 한번 돌린 후 결과 index user  column search_index**

In [28]:
search_prediction_

searchid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,205583,205661,205799,205945,205967,206068,206135,206208,206272,206293,206385,206435,206499,206523,206714,206727,206777,206805,206845,206861,206899,206949,207023,207111,207203,207213,207309,207311,207367,207405,207642,207830,207890,208002,208080,208112,208737,208793,208939,209163
1,3.74027,3.74038,3.74048,3.74067,3.74051,3.74055,3.74057,3.74071,3.74066,3.74043,3.7405,3.74062,3.74069,3.74064,3.74065,3.74058,3.74054,3.74062,3.74001,3.74065,3.74049,3.7406,3.74065,3.7406,3.74046,3.74068,3.7407,3.7407,3.74066,3.74071,3.7406,3.7404,3.74072,3.7402,3.7407,3.74056,3.74072,3.74071,3.74037,3.74072,...,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072,3.74072
2,3.54664,3.54658,3.54658,3.54662,3.54659,3.54664,3.54659,3.54665,3.54661,3.54664,3.54661,3.54663,3.54665,3.54662,3.54663,3.54664,3.54661,3.54661,3.54646,3.54662,3.5466,3.5466,3.54662,3.5466,3.54656,3.54665,3.54665,3.54665,3.54665,3.54665,3.54662,3.54659,3.54666,3.54655,3.54665,3.54663,3.54666,3.54665,3.54652,3.54665,...,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666,3.54666
3,3.68664,3.68661,3.68662,3.68664,3.68662,3.68665,3.68663,3.68665,3.68663,3.68662,3.68663,3.68663,3.68664,3.68664,3.68664,3.68665,3.68664,3.68664,3.68656,3.68664,3.68664,3.68663,3.68663,3.68663,3.68663,3.68664,3.68665,3.68665,3.68665,3.68665,3.68663,3.68667,3.68665,3.68661,3.68665,3.68664,3.68665,3.68665,3.68661,3.68665,...,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665,3.68665
4,3.27969,3.27967,3.27972,3.27973,3.27971,3.27974,3.27971,3.27974,3.27973,3.27971,3.27972,3.27973,3.27973,3.27973,3.27973,3.27975,3.27972,3.27973,3.27959,3.27973,3.27971,3.27972,3.27974,3.27972,3.27969,3.27975,3.27974,3.27975,3.27973,3.27974,3.27971,3.27972,3.27975,3.27968,3.27974,3.27972,3.27975,3.27975,3.27968,3.27975,...,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975,3.27975
5,3.77413,3.77413,3.77415,3.77425,3.77415,3.77422,3.77419,3.77427,3.77422,3.77419,3.77419,3.77421,3.77427,3.77422,3.77423,3.77424,3.77416,3.77423,3.77393,3.77424,3.77412,3.77422,3.77426,3.77421,3.77411,3.77426,3.77427,3.77426,3.77425,3.77427,3.77423,3.77428,3.77428,3.77393,3.77428,3.77421,3.77428,3.77428,3.77383,3.77428,...,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428,3.77428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7041,4.08729,4.08745,4.08752,4.08768,4.08754,4.087